_Softwarepraktikum Blockkurs März 2021 (WS2020/21)_ 

# Einführung in Numerische Programmierung[<sup>1</sup>](#fn1)

[Jörn Behrens](https://www.math.uni-hamburg.de/numgeo/) (joern.behrens@uni-hamburg.de)

## 1. Einführung und Ziel

In diesem Notebook wird die Numerische Programmierung eingeführt. Zunächst werden wir über Fließkommazahlen und deren Eigenschaften reflektieren. Anschließend lernen wir verschiedene grundlegende Python Konstrukte kennen, die im Umgang mit numerischen Berechnungen wichtig sind.

Wir werden die Einführung in die NumPy Arrays aus dem Notebook [06 - Visualization](06%20-%20Visualization%20with%20Matplotlib.ipynb) an dieser Stelle nicht wiederholen, sie gehören aber natürlich zu den Voraussetzungen für dieses Notebook.

## 2. Fließkommazahlen

In der numerischen Programmierung ist es wichtig zu verstehen, dass viele der Rechenregeln nur näherungsweise gelten. Der Computer speichert Zahlen als endliche binäre Ziffernreihen. Insbesondere irrationale Zahlen, die eigentlich unendliche Dezimalzahlen darstellen, können als nur näherungsweise im Computer dargestellt werden.

Hier ein paar Beispiele:

In [ ]:
import numpy as np
from convtool import *
print('Ein Zehntel Fließkomma : ',1/10,' und Binär: ',float2bin(1/10))
print('Kreiszahl pi Fließkomma: ',np.pi,' und Binär: ',float2bin(np.pi))
print('Die Null Fließkomma    : ',0.0,' und Binär: ',float2bin(0.0))
print('Die Eins Fließkomma    : ',1.0,' und Binär: ',float2bin(1.0))

An den Beispielen sieht man, dass schon so einfache Zahlen wie $\frac{1}{10} = 0.1$ nicht als endliche Binärzahl gespeichert werden können und daher nicht exakt im Computer repräsentiert sind.

Die Abweichung, die wir in der Regel erwarten können ist durch die Maschinengenauigkeit $\epsilon$ gegeben, die wir immer im normierten Interval, also bei der 1, ermitteln. Diese Zahl kann mit der NumPy Methode `eps` ermittelt werden:

In [ ]:
print('Maschinengenauigkeit für standard float Typ: ',np.finfo(float).eps)
print('Maschinengenauigkeit für 32 bit float Typ  : ',np.finfo(np.float32).eps)

### Rundungsfehler

Wenn wir wiederholt Berechnungen mit Rundungsfehler ausführen, kann das schon recht schnell zu sichtbaren Fehlern führen. Das demonstriere ich hier: Wir erstellen ein Array mit 101 Einträgen von 0 bis 10 mit Schrittweiten 0.1. Im ersten Fall verwende ich einfach ein Array Konstrukt, im zweiten Fall berechne ich die einzelnen Einträge, indem ich jeweils 0.1 zum vorherigen Ergebnis aufaddiere. Schließlich vergleiche ich beide Einträge am Ende des Vektors (also jeweils den Eintrag, der den Wert 10 haben sollte):

In [ ]:
# generiere ein array von 0..10
x1 = np.arange(0,10.1,0.1)

# generiere ein analoges array indem jeweils 1/10 addiert wird
x2 = np.arange(0.0)
x2= np.append(x2,[0.0])
for i in range(1,101):
    t = x2[i-1]+ 0.1
    x2 = np.append(x2,[t])

print('x1[-1] : ',x1[-1])
print('x2[-1] : ',x2[-1])
print('Differenz x1-x2 : ',(x1[-1] - x2[-1]))

Hier haben wir 100 Operationen ausgeführt und damit einen Fehler in der 14. Nachkommastelle erzeugt, wobei die Darstellung von $\frac{1}{10}$ auf 16 Nachkommastellen genau ist. Wir haben also - logischerweise - in $10^2$ Operationen zwei Dezimalstellen an Genauigkeit verloren.

Manchmal kann man aber schon in einer Operation einen signifikanten Unterschied erzielen:

In [ ]:
diff3 = np.abs(3. - (0.3/0.1))
print('3 - 0.3/0.1 in Fließkommaoperationen: ', diff3)

**Bemerkung:** Wir behalten dieses Verhalten bei numerischen Berechnungen jetzt im Hinterkopf. In der Numerischen Mathematik geht es also vor allem darum, Berechnungen auf dem Computer so durchzuführen, dass diese Rundungsfehler die erwarteten Ergebnisse nicht mehr verfälschen, als unvermeidbar ist.

## 3. Erzeugung von Arrays

Im folgenden werden wir eine Reihe von funktionen kennen lernen, mit deren Hilfe wir uns Daten erzeugen können. Häufig benötigen wir einfache Gitter, Folgen von reellen (aka Fließkomma-) Zahlen, einfache Matrizen.

Hier sind eine Reihe von Befehlen, die Nützlich sind:

* `arange` erzeugt allgemein einen Vektor mit konsekutiven Werten
* `array` erzeugt ein array ausgegebenen Werten
* `matrix` erzeugt ein objekt des Typs Matrix aus gegebenen Werten
* `eye`, `ones`, `zeros`, `tri` erzeugen spezielle Vektoren oder Matrizen: Einheitsmatrix, Einsen, Nullen, (untere) Dreiecksmatrix
* `diag`, `triu`, `tril` extrahieren Diagonale, untere oder obere Dreiecksmatrizen

Hier sind entsprechende Beispiele:

In [ ]:
vec = np.arange(20)               # erzeugt Vektor der Länge 20 mit Einträgen 0..19.
Id1 = np.matrix('1. 0.; 0. 1.')   # erzeugt 2x2-Einheitsmatrix (Typ matrix)
Id2 = np.array([[1.,0.],[0.,1.]]) # erzeugt 2x2-Einheitsmatrix (Typ array)
Id3 = np.eye(2)                   # erzeugt ebenfalle 2x2-Einheitsmatrix (Typ array)
print('arange:\n',vec)
print('matrix:\n',Id1)
print('array:\n',Id2)
print('eye:\n',Id3)

In [ ]:
m = np.matrix('3,2,1;4,5,6;9,8,7')
print('Matrix m:\n', m)
print('Diagonale:\n', np.diag(m))
print('Obere Dreiecksmatrix:\n', np.triu(m))

### linspace und meshgrid

Wir hatten die Befehle`linspace` und `meshgrid` schon in der Visualisierung kennen gelernt. Hier wollen wir sie noch einmal genauer betrachten. `linspace` erzeugt eine Reihe von "Gitterpunkten" mit definierter Anfangs- und Endkoordinate, die gleichmäßig über das gegebene Intervall verteilt sind. Per default werden 50 solcher Punkte erzeugt. Hier ist ein Beispiel für das Einheitsintervall:

In [ ]:
x = np.linspace(0.,1.)
print('Länge von x: ',x.size)
print('Erste zwei und letzte zwei Einträge von x:\n',x[:2],' [...] ',x[-2:])

Mit dem Befehl `meshgrid` lässt sich nun ein mehrdimensionales Gitter erzeugen. Wir werden das hier für ein zwei-dimensionales Gitter vorführen, wobei wir in beiden dimensionen die gleiche Koordinateneinteilung des Einheitsintervalls (wie oben) verwenden. Die Rückgabevektoren sind jeweils Arrays der Größe (`x.size` $\times$ `x.size`), wobei die X-Koordinaten jeweils in y-Richtung wiederholt werden und die Y-Koordinaten in x-Richtung.

In [ ]:
X,Y = np.meshgrid(x,x)
X.shape

## 4. Vordefinierte Funktionen

In NumPy gibt es eine große Menge vorgefertigter Funktionen für alle möglichen Zwecke. Natürlich können wir trigonometrische Funktionen verwenden, ebenso die Exponentialfunktion und Logarithmen. Wichtig ist hier, dass die Funktionen nicht nur auf einzelne Werte anwendbar sind, sondern elementweise auf ganze Vektoren oder Matrizen. Das haben wir auch in der Einführung der Visualisierung schon verwendet. Hier daher nur ein paar wenige Beispiele. Diese Klasse von Funktionen nennen sich _Universal Funktions_ (`ufunc`) und es findetn sich über 60 solcher Funktionen in NumPy.

* spezielle Zahlen: Unendlich - `inf`, Not a Number - `nan`
* einfache Operationen: `add` oder `+`, `multiply` oder `*`, 
* trigonometrische Funktionen: `sin`, `cos`, `tan`, `arcsin`,`sinh`,`radians`, etc.
* Rundungen und Vorzeichen: `fabs`, `sign`, `rint`, `floor_divide`, etc.
* Exponentialfunktion, Potenzen und Logarithmus: `exp`, `exp2`, `power`, `square`, `sqrt`, `log`, `log10`, etc.
* Vergleich: `greater` oder `>`, `less` oder `<`, `equal` oder `==`, `maximum`, `minimum`, etc.

In [ ]:
print('Sinus einer einzelnen Zahl sin(pi/2):\n', np.sin(np.pi/2))
z=np.arange(5)*np.pi/4 + np.pi
print('Sinuns angewandt auf Vektor sin(z):\n', np.sin(z))
print('Beträge der eben berechneten Werte:\n', np.fabs(np.sin(z)))
print('Elementweises Maximum von z und sin(z):\n', np.maximum((z-np.pi),np.sin(z)))
print('El. Minimum von[0 1 inf] und [nan inf nan]:\n', np.minimum([0,1,np.inf],[np.nan,np.inf,np.nan]))

### Fouriertransformation

Die Schnelle Fouriertransformation, eingeführt von [James W. Cooley und John W. Tuckey (1965)](https://doi.org/10.1090/S0025-5718-1965-0178586-1), ist ein "game-changing" Algorithmus. Die Implementation findet sich in Numpy natürlich in einer schnellen, maschinen-optimierten Version.

* Fourier-Transformation in 1D: `fft`, inverse Fourier-Transformation in 1D: `ifft`
* das gleiche in 2D: `fft2`, `ifft2`

Diese Funktionen werden Sie für die eigene Implementation benötigen und es wird dazu ein eigenes Blatt geben, daher verzichten wir hier auf die Beispiele.

**Bemerkung:** Eine erweiterte Funktionalität und noch schnellere implementierte Algorithmen finden sich in  SciPy. Die Erweiterungen betreffen bestimmte Eigenschaften der Daten, beispielsweise reelle Daten, Hermitische (symmetrische) Daten, ein- bis mehr-dimensionale Daten. Auch diskrete Sinus- und Cosinus-Transformationen sind verfügbar. Hier eine Auswahl

* `fft`, `fft2`, `fftn`, `rfft`, `hfft` für die Vorwärts-Transformation
* `ifft`, `ifft2`, `ifftn`, `irfft`, `ihfft` für die inverse Transformation
* `dct`, `dctn`, `dst`, `dstn` für die Cosinus- und Sinus-Transformation
* `idct`, `idctn`, `idst`, `idstn` für die Inversen dazu

### Lineare Algebra

Das Arbeiten mit Matrizen und Vektoren und entsprechende Funktionen der angewandten Linearen Algebra sind eine Stärke von NumPy. Dazu gibt es eine Vielzahl von Funktionen, von denen hier einige wichtige vorgestellt werden. Einige dieser Funktionen sind im Untermodul `linalg` enthalten.

* Vektorprodukte: `dot`, `inner`, `outer`
* Matrixprodukte: `matmul`, `linalg.matrix_power`
* Zerlegungen (linalg): `qr`, `cholesky`, `svd`
* Eigenwerte (linalg): `eig`, `eigvals`
* Normen und Werte (linalg): `det`, `cond`, `norm`, `matrix_rank`, `trace`

Hier ein paar Anwendungen:

In [ ]:
# generiere eine (5x5) Zufallsmatrix
A = np.random.rand(5,5)
print('Matrix A und A*A:\n', A, '\n\n', np.matmul(A,A))
print('Eigenwerte der Matrix:\n',np.linalg.eigvals(A))
Q,R = np.linalg.qr(A)
print('QR-Zerlegung der Matrix:\nQ:\n',Q,'\nR:\n',R)
print('Kondition und Rang der Matrix: ', np.linalg.cond(A),' , ', np.linalg.matrix_rank(A))

### Lineare Algebra im Modul SciPy

Das Modul **SciPy** enthält eine Vielzahl weiterer Funktionen, die über die in NumPy enthaltenen hinausgehen. Insbesondere für die in `linalg` enthaltenen Funkionen bietet SciPy eine Reihe von Vorteilen. Zum einen gibt es weiter über NumPy hinausgehende Funktionen. Als Beispiel

| NumPy linalg | SciPy linalg |
|:-------------|:-------------|
| qr | qr |
| svd | svd |
| cholesky | cholesky |
| - | lu |
| - | shur |
| - | interpolative |

Hier ist eine Auswahl an SciPy Untermodulen, die für viele numerische Berechnungen sehr hilfreich sind.

* Spezielle Funktionen `cipy.special`
* Integration und Lösung von gewöhnlichen Differentialgleichungen `scipy.integrate`
* Optimierung `scipy.optimize`
* Interpolation `scipy.interpolate`
* Fourier Transformation `scipy.fft`
* Signal Verarbeitung `scipy.signal`
* Lineare Algebra `scipy.linalg`
* Räumliche Datenstrukturen und Algorithmen `scipy.spatial`
* Statistik `scipy.stats`
* Mehrdimensionale Bildverarbeitung `scipy.ndimage`

Wir werden in einem folgenden Notebook auf einige dieser Fähigkeiten zurück kommen.

### Beispiel Cholesky Faktorisierung

Sie kennen wahrscheinlich den Gauß-Algorithmus zur Lösung linearer Gleichungssysteme. Die moderne Form für allgemeine Matrizen ist eine $LU$-Zerlegung in eine obere und untere Dreiecksmatrix, die dann jeweils mit einer Vorwärts- bzw. Rückwärtssubstitution gelöst werden kann. In mathematischer Sprache ausgedrückt
$$
Ax = b\quad\textsf{wird\ ersetzt\ durch}\quad LUx = b,
$$
wobei $A=LU$ ist. Anschließend löst man die beiden Dreiecks-Systeme
\begin{align}
Ly &= b,\\
Ux &= y.
\end{align}
Ist die Matrix symmetrisch positiv definit (s.p.d.), dann kann man eine Vereinfachung der $LU$-Zerlegung, nämlich die *Cholesky-Zerlegung* verwenden, die in ihrer Struktur schon die Symmetrie der Matrix annimmt und auch aus der positiv Definitheit schließt, dass kein Pivot gesucht werden muss. 

Um das folgende Beispiel auszuführen, benötigen wir eine Reihe von Service-Routinen, die ich hier importiere.

In [ ]:
from time import time
from numpy import shape, zeros, transpose, sqrt, dot, matrix, array, tril
from numpy.linalg import norm, cholesky
from numpy.random import rand
from scipy.linalg import cholesky as scholesky

Ich möchte den Algorithmus hier gar nicht genauer erklären, sondern auf einen anderen Aspekt eingehen, die Performanz. Schreibt man den Algorithmus Zeile für Zeile auf, so lautet er

In [ ]:
def cholesky1(A):
    [n,n] = shape(A)
    L = zeros((n,n))
    for j in range(n):
        L[j,j] = sqrt( A[j,j] - dot(L[j,0:j],transpose(L[j,0:j])) )
        for i in range(j+1,n):
            L[i,j] = ( A[i,j] - dot(L[i,0:j],transpose(L[j,0:j])) )/L[j,j]
    return L

Um die Effizienz in Python zu steigern, vermeidet man nach Möglichkeit `for`-Schleifen. Wenn wir die innere Schleife durch eine Vektor-Notierung ersetzen, dann entspricht das einer Matrix-Vektor-Operation, die durch eine effiziente interne Funktion ersetzt werden kann:

In [ ]:
def cholesky2(A):
    [n,n] = shape(A)
    L = zeros((n,n))
    for j in range(n):
        L[j,j] = sqrt( A[j,j] - dot(L[j,0:j],transpose(L[j,0:j])) )
        L[j+1:n,j] = ( A[j+1:n,j] - dot(L[j+1:n,0:j+1],transpose(L[j,0:j+1])) )/L[j,j]
    return L

Schließlich kann auch die NumPy oder auch die SciPy `cholesky` Implementierung aufgerufen werden. Um diese Aufrufe alle gegeneinander auf ihre Laufzeiteigenschaften zu testen, benötigen wir eine Zeitmessung und eine Funktion, die uns s.p.d. Matrizen erzeugt. Beides habe ich unten definiert.

In [ ]:
class Timer(object):
    """gives analogue results as matlabs tic and toc function
    usage:
    with Timer():
    ###do some stuff here###
    """
    def __init__(self, disp=True):
        self.disp = disp

    def __enter__(self):
        self.tic = time()

    def __exit__(self, type, value, traceback):
        self.toc = time() - self.tic
        if self.disp:	print('Elapsed time is %6.6f' % (self.toc) + ' seconds')

In [ ]:
def spd(n):
    B = rand(n,n)
    A = dot(B,transpose(B))
    return A

Nun schauen wir uns an, wie schnell die jeweilige Funktion eine Zerlegung eine ($1000\times 1000$)-Matrix bewerkstelligt:

In [ ]:
# Größe der Matrix
n = 1000
A = spd(n)
  
print('Level 1 BLAS Cholesky factorization')
with Timer(): L1=cholesky1(A)
  
print('Level 2 BLAS Cholesky factorization')
with Timer(): L2=cholesky2(A)

print('Level 3 BLAS Cholesky factorization from numpy')
with Timer(): LL=cholesky(A)

print('Level 3 BLAS Cholesky factorization from scipy')
with Timer(): LS=scholesky(A,lower=True)

Es gibt also erhebliche Unterschiede, die naive Implementierung ist der SciPy Version um einen Faktor 140 unterlegen! Alle Funktionen führen die selbe Operation aus! Das können wir testen, indem wir die Normen der Matrizen miteinander vergleichen. Wir nehmen dazu an, dass die fertig implementierten Funktionen `cholesky` aus SciPy und NumPy richtig rechnen:

In [ ]:
print('Relativer Fehler von BLAS Level 1 Cholesky   :' + '%12.9e' % (norm(LL-L1)/norm(LL)))
print('Relativer Fehler von BLAS Level 1 Cholesky   :' + '%12.9e' % (norm(LL-L2)/norm(LL)))
print('Relativer Fehler von SciPy und NumPy Cholesky:' + '%12.9e' % (norm(LL-LS)/norm(LL)))

## 5. Sortieren und Finden

NumPy stellt eine Reihe von sehr hilfreichen Funktionen zur Verfügung, die es einem ermöglichen, in großen Vektoren oder Matrizen einzelne Einträge zu finden, oder auch Vektoren zu sortieren.

* Sortieren: `sort`, `msort`, `lexsort`, `sort_complex`
* Finden: `argmax`, `argmin`, `argwhere`, `where`, `nonzero`, `extract`

Wieder ein paar Beispiele. Zunächst erzeugen wir uns einen Vektor, auf den wir dann die Methoden anwenden.

In [ ]:
a = np.array([1, 2, 3, 0, 7, 6, 5, np.inf, 4])
print('Vektor a sortiert:\n',np.sort(a))
print('Index des Maximums im Vektor: ',np.argmax(a), ' und dessen Wert: ',a[np.argmax(a)])
print('Index des Minimums im Vektor: ',np.argmin(a), ' und dessen Wert: ',a[np.argmin(a)])
i = np.where(a==5) # Rückgabewert ist immer ein array, daher werten wir i aus.
print('Index an dem a den Wert 5 annimmt: ',i[0])
print('Vektor aller Werte größer 3:\n',np.extract(a>3,a))

## 6. Einlesen und Schreiben von einfachen Dateien

Zum Einlesen von Daten oder auch speichern von Daten lässt sich sehr viel sagen. Python ist ein extrem mächtiges Werkzeug, um alle möglichen Datenformate von allen möglichen Orten zu lesen, zu konvertieren, zu verarbeiten und wieder zu speichern. Das würde weit über den Rahmen dieses Kurses hinaus gehen.

Daher werden wir es hier bei einfachen Funktionen zum Lesen und Schreiben von ASCII (also Text) Dateien bewenden lassen. Die beiden wesentlichen Tools dafür sind

* `loadtxt`
* `savetxt`

Wir werden das kurz an einem Beispiel zeigen:

In [ ]:
np.savetxt('a_save.txt',a)
np.savetxt('LS_save.txt.gz',LL)

Als nächstes schauen wir uns an, welche Dateien im Verzeichnis gelandet sind. Das geschieht in der Ausführungszeile mit einem vorangestellten `!`-Zeichen. Auf dem Mac oder unter Linux kann man sich die Dateien mit dem Befehl `ls` (für list) ansehen. Unter Windows sollte das mit dem Befehl `dir` gehen. Die Matrix $LL$ hat immerhin 1000 $\times$ 1000 = 1 Mio. Einträge, daher ist sie als komprimierte (.gz) Datei geschrieben worden (_ob das unter Windows so funktioniert, weiß ich nicht..._). Den `cat` Befehl verwende ich, um den Inhalt der reinen txt-Datei anzuzeigen (_das geht unter Windows wahrscheinlich nicht_).

In [ ]:
!ls -l *.txt*
!cat a_save.txt

Nun lesen wir diese Dateien wieder ein:

In [ ]:
a_new = np.loadtxt('a_save.txt')
print('Eingelesenes a:\n',a_new)
L_new = np.loadtxt('LS_save.txt.gz')
print('Erste 10 Sub-Diagonalelemente des eingelesenen L:\n',np.diag(L_new,-1)[:10])
print('Zum Vergleich die Elemente des Original LS:\n',np.diag(LS,-1)[:10])

## 7. Aufgaben

1. **Aufgabe (Hilbertmatrix)**: Erzeugen sie mittels des Befehls `hilbert` aus dem Paket SciPy.linalg eine Hilbertmatrix der Größe $5\times 5$. Überprüfen Sie die Kondition der Matrix und berechnen sie die Matrix-Norm. Weiter berechnen Sie die LU-Zerlegung und die Cholesky-Zerlegung. Beide unterscheiden sich, unter anderem, weil die LU-Zerlegung eine Spalten-Permutation während der Berechnung vornimmt.

2. **Aufgabe (Arrays und Fuktionen)**: 
 * Erzeugen Sie einen Vektor aus Zufallszahlen der Länge 100. 
 * Erzeugen Sie einen zweiten Vektor aus absteigenden ganzen Zahlen von 99-0 (also ebenfalls der Länge 100).
 * Berechnen Sie das Skalarprodukt beider Vektoren.
 * Skalieren Sie den zweiten Vektor mit 10 und berechnen Sie den Mittelwert (recherchieren Sie, welche Funktion sich dafür eignet)

---
1) <span id="fn1">Copyright Notice:
    
    Einführung in Numerische Programmierung, Copyright (C) 2020  Jörn Behrens
    
        Prof. Dr. Jörn Behrens
        Universität Hamburg, Dept. Mathematik
        Bundesstrasse 55
        20146 Hamburg, Germany
        joern.behrens@uni-hamburg.de

    This program is free software; you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation; either version 2 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License along
    with this program; if not, write to the Free Software Foundation, Inc.,
    51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.


</span>